In [1]:
import pandas as pd
import plotly.express as px
import os
import glob

%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

UsageError: Line magic function `%%javascript` not found.


In [2]:
# input data directory
# I have downlaoded the data from Kaggle Competition and unzip using 7z tool in ubuntu command terminal
# TODO: Use the Kaggle API to access the data
data_dir = '/home/ibbu/PycharmProjects/retail_sales_prediction/data/'

In [3]:
class readDataStore():
    """
    This module is to read the data from data store.
    
    In our case, our date store is local and is in .csv format
    """
    
    def __init__(self, data_dir):
        
        self.data_dir = data_dir
    
    def read_stores(self, file_name):
        
        df = pd.read_csv(self.data_dir + file_name)
        
        return df
    
    def read_items(self, file_name):
        
        df = pd.read_csv(self.data_dir + file_name)
        
        return df
    
    def read_transactions(self, file_name):
        
        df = pd.read_csv(self.data_dir + file_name)
        
        return df
    
    def read_oil(self, file_name):
        
        df = pd.read_csv(self.data_dir + file_name)
        
        return df
    
    def read_holidays(self, file_name):
        
        df = pd.read_csv(self.data_dir + file_name)
        
        return df
    
    def read_train(self, file_name):
        
        df = pd.read_csv(self.data_dir + file_name)
        
        return df
    
    def read_test(self, file_name):
        
        df = pd.read_csv(self.data_dir + file_name)
        
        return df
    
        
        

readDataStore = readDataStore(data_dir)

# Oil Prices

One of the esternal data given is the daily oil prices. Let us look at this time series and make our initial observations. 

In [4]:
df_oil = readDataStore.read_oil('oil.csv')

In [5]:
df_oil.count()

date          1218
dcoilwtico    1175
dtype: int64

The oil prices have some missing values, It could be due to trading holidays or weekend. 

In [9]:
px.line(df_oil, x= 'date', y='dcoilwtico')

By looking at the price chart, it seems like there is data issue and it needs further investigation. For time being, we can simply forwardfill with previous price. 

In [10]:
px.line(df_oil.fillna(method = 'ffill'), x= 'date', y='dcoilwtico')

In [13]:
df_oil = df_oil.set_index('date')
df_oil_data = pd.DataFrame()
df_oil_data['dcoilwtico'] = df_oil['dcoilwtico']
df_oil_data['dcoilwtico_ffill'] = df_oil['dcoilwtico'].fillna(method = 'ffill')
df_oil_data['dcoilwtico_ma20'] =df_oil['dcoilwtico'].fillna(method = 'ffill').rolling(window=20).mean()

In [19]:
df_oil_data = df_oil_data.stack().reset_index().rename(columns = {'level_1':'price_type', 0:'oil_price'})

In [20]:
px.line(df_oil_data, x = 'date', y='oil_price', color='price_type')

# Holidays Events

Holidays are special events and model should be able to handle this. Apart from holidays, there are others special days like christmas or local events as we have seen in transactions plot. 

In [21]:
df_holidays = readDataStore.read_oil('holidays_events.csv')

In [22]:
df_holidays.count()

date           350
type           350
locale         350
locale_name    350
description    350
transferred    350
dtype: int64

In [30]:
df_holidays_count = df_holidays.groupby(['type', 'locale', 'locale_name'])['date'].count()

In [32]:
df_holidays_count = df_holidays_count.reset_index().rename(columns = {'date':'count'})

In [35]:
df_holidays_count.head()

,type,locale,locale_name,count
0,Additional,Local,Guayaquil,5
1,Additional,Local,Quito,6
2,Additional,National,Ecuador,40
3,Bridge,National,Ecuador,5
4,Event,National,Ecuador,56


In [33]:
px.bar(df_holidays_count, x = 'type', y='count', color = 'locale')

It is important to understand the holidays data frame and incorporate in out model somehow.

TODO: Think how to use this information to our model